# Archived

In [9]:
import pandas as pd
claims_dev = pd.read_json(path_or_buf="data/claims_dev.jsonl", lines=True)
claims_dev

,id,claim,evidence,cited_doc_ids
0,1,0-dimensional biomaterials show inductive prop...,{},[31715818]
1,3,"1,000 genomes project enables mapping of genet...","{'14717500': [{'sentences': [2, 5], 'label': '...",[14717500]
2,5,1/2000 in UK have abnormal PrP positivity.,"{'13734012': [{'sentences': [4], 'label': 'SUP...",[13734012]
3,13,5% of perinatal mortality is due to low birth ...,{},[1606628]
4,36,A deficiency of vitamin B12 increases blood le...,{},"[5152028, 11705328]"
...,...,...,...,...
295,1379,Women with a higher birth weight are more like...,"{'16322674': [{'sentences': [5], 'label': 'SUP...","[16322674, 27123743, 23557241, 17450673]"
296,1382,aPKCz causes tumour enhancement by affecting g...,"{'17755060': [{'sentences': [3], 'label': 'CON...",[17755060]
297,1385,cSMAC formation enhances weak ligand signalling.,"{'306006': [{'sentences': [4], 'label': 'SUPPO...",[306006]
298,1389,mTORC2 regulates intracellular cysteine levels...,"{'23895668': [{'sentences': [2, 3], 'label': '...",[23895668]


In [10]:
idx = 5
claims_dev["claim"].iloc[idx]

'A high microerythrocyte count raises vulnerability to severe anemia in homozygous alpha (+)- thalassemia trait subjects.'

In [11]:
claims_dev["evidence"].iloc[idx]

{'18174210': [{'sentences': [1, 9], 'label': 'CONTRADICT'},
  {'sentences': [10], 'label': 'CONTRADICT'}]}

In [12]:
claims_dev["cited_doc_ids"].iloc[idx]

[18174210]

In [13]:
# Corpus
corpus = pd.read_json(path_or_buf="data/corpus.jsonl", lines=True)
corpus.head()

,doc_id,title,abstract,structured
0,4983,Microstructural development of human newborn c...,[Alterations of the architecture of cerebral w...,False
1,5836,Induction of myelodysplasia by myeloid-derived...,[Myelodysplastic syndromes (MDS) are age-depen...,False
2,7912,"BC1 RNA, the transcript from a master gene for...",[ID elements are short interspersed elements (...,False
3,18670,The DNA Methylome of Human Peripheral Blood Mo...,[DNA methylation plays an important role in bi...,False
4,19238,The human myelin basic protein gene is include...,[Two human Golli (for gene expressed in the ol...,False


In [14]:
corpus[corpus["doc_id"] == 13734012]["title"].values

array(['Prevalent abnormal prion protein in human appendixes after bovine spongiform encephalopathy epizootic: large scale survey'],
      dtype=object)

In [15]:
for s in corpus[corpus["doc_id"] == 13734012]["abstract"].iloc[0]:
    print(s)

OBJECTIVES To carry out a further survey of archived appendix samples to understand better the differences between existing estimates of the prevalence of subclinical infection with prions after the bovine spongiform encephalopathy epizootic and to see whether a broader birth cohort was affected, and to understand better the implications for the management of blood and blood products and for the handling of surgical instruments.   

DESIGN Irreversibly unlinked and anonymised large scale survey of archived appendix samples.   

SETTING Archived appendix samples from the pathology departments of 41 UK hospitals participating in the earlier survey, and additional hospitals in regions with lower levels of participation in that survey.   
SAMPLE 32,441 archived appendix samples fixed in formalin and embedded in paraffin and tested for the presence of abnormal prion protein (PrP).   

RESULTS Of the 32,441 appendix samples 16 were positive for abnormal PrP, indicating an overall prevalence 

In [16]:
len(corpus), corpus.columns

(5183, Index(['doc_id', 'title', 'abstract', 'structured'], dtype='object'))

In [17]:
import numpy as np
import tiktoken
encoder = tiktoken.encoding_for_model("text-embedding-ada-002")

len_tokens = list()
for i in range(len(corpus)):
    text = ""
    for s in corpus["abstract"].iloc[i]:
        text += s + " "

    tokens = len(encoder.encode(text))
    len_tokens.append(tokens)

In [19]:
np.array(len_tokens).sum() * (0.0001 / 1000)

0.15598320000000002

In [20]:
train = pd.read_json("data/claims_train.jsonl", lines=True)

ids = list()
banyak_kontra = 0
for i in range(len(train)):
    doc_ids = train["evidence"].iloc[i].keys()

    labels = list()
    for evid in doc_ids:
        for j in range(len(train["evidence"].iloc[i][evid])):
            l = train["evidence"].iloc[i][evid][j]["label"]
            labels.append(l)
        
        if "CONTRADICT" not in labels:
            ids.append(i)
        else:
            banyak_kontra += 1

In [21]:
testing_data = train.iloc[ids]

In [19]:
# !pip install pinecone-client -qq

In [25]:
import pinecone      
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

pinecone.init(      
	api_key='f09dbe82-3f3a-4edb-8360-2ae2de01603d',      
	environment='gcp-starter'      
)      
index = pinecone.Index('scifact')
embedder = OpenAIEmbeddings()

In [31]:
abstracts = list()

for a in corpus["abstract"]:
    text = ""
    for s in a:
        text += s + " "
    abstracts.append(text)

In [53]:
embeddings = embedder.embed_documents(abstracts)

In [109]:
vectors = list()
for i in range(len(corpus)):
# for i in range(5):
    vectors.append(
        {
        "id" : f"ABS{str(i)}",
        "values" : embeddings[i],
        "metadata" : {
            "doc_id" : str(corpus["doc_id"].iloc[i]),
            "title" : corpus["title"].iloc[i],
            # "text" : abstracts[i]
        }
        }
    )

In [ ]:
upsert_response = index.upsert(vectors=vectors)

In [87]:
upsert_response

{'upserted_count': 370}

In [88]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0037,
 'namespaces': {'': {'vector_count': 370}},
 'total_vector_count': 370}

In [73]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index("scifact")

vectorstore = Pinecone(
    index, embedder, text_field
)

retriever = vectorstore.as_retriever()

In [75]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

llm = OpenAI()

In [76]:
# Multiquery Retriever
from langchain.retrievers.multi_query import MultiQueryRetriever

query = testing_data["claim"].iloc[0]

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=llm
)
with get_openai_callback() as cb:
    contexts = retriever_from_llm.get_relevant_documents(query=query, k=3)
cb

Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.
Found document with no `text` key. Skipping.


Tokens Used: 164
	Prompt Tokens: 108
	Completion Tokens: 56
Successful Requests: 1
Total Cost (USD): $0.00328

In [3]:
from langchain.vectorstores import Chroma
import chromadb

In [ ]:
chroma_client = chromadb.PersistentClient()
collection = chroma_client.create_collection(name="scifact_benchmark")

In [52]:
sample_corpus = corpus.sample(5, random_state=42)
sample_corpus

,doc_id,title,abstract,structured
4678,39462488,Replisome stalling and stabilization at CGG re...,[Expanded CGG repeats cause chromosomal fragil...,False
1432,7975937,Cyclooxygenase-Dependent Tumor Growth through ...,[The mechanisms by which melanoma and other ca...,False
410,2211868,"Epidemiological expansion, structural studies,...",[β-Lactamase evolution presents to the infecti...,False
3869,26079071,Crizotinib in ROS1-rearranged non-small-cell l...,[BACKGROUND Chromosomal rearrangements of the ...,True
5068,67787658,Cycling Quiescence in Temozolomide Resistant G...,[Glioblastoma multiforme (GBM) is a fatal mali...,False


In [53]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.callbacks import get_openai_callback

embedder = OpenAIEmbeddings()

abstracts = list()

for a in sample_corpus["abstract"]:
    text = ""
    for s in a:
        text += s + " "
    abstracts.append(text)
len(abstracts)

5

In [55]:
embeddings = embedder.embed_documents(abstracts)

In [56]:
metadatas = list()
for i in range(len(sample_corpus)):
    metadatas.append({
        "doc_id" : str(sample_corpus["doc_id"].iloc[i]),
        "title": sample_corpus["title"].iloc[i]
    })

ids = [f"abs{str(i)}" for i in range(len(sample_corpus))]
metadatas[:2], ids[:2]

([{'doc_id': '39462488',
   'title': 'Replisome stalling and stabilization at CGG repeats, which are responsible for chromosomal fragility'},
  {'doc_id': '7975937',
   'title': 'Cyclooxygenase-Dependent Tumor Growth through Evasion of Immunity'}],
 ['abs0', 'abs1'])

In [57]:
collection.add(
    embeddings=embeddings,
    documents=abstracts,
    metadatas=metadatas,
    ids=ids
)

In [59]:
langchain_chroma = Chroma(
    client=chroma_client,
    collection_name="scifact_benchmark",
    embedding_function=embedder,
)

In [60]:
print("There are", langchain_chroma._collection.count(), "in the collection")

There are 5 in the collection


In [62]:
docs = langchain_chroma.similarity_search_with_score("chromosome")

# We're ready to go

In [110]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import chromadb
import pandas as pd
from tqdm import tqdm
from langchain.callbacks import get_openai_callback

In [8]:
# Corpus
corpus = pd.read_json(path_or_buf="data/corpus.jsonl", lines=True)

abstracts = list()
for a in corpus["abstract"]:
    text = ""
    for s in a:
        text += s + " "
    abstracts.append(text)

# Testing Data
train = pd.read_json("data/claims_train.jsonl", lines=True)

ids = list()
banyak_kontra = 0
for i in range(len(train)):
    doc_ids = train["evidence"].iloc[i].keys()

    labels = list()
    for evid in doc_ids:
        for j in range(len(train["evidence"].iloc[i][evid])):
            l = train["evidence"].iloc[i][evid][j]["label"]
            labels.append(l)
        
        if "CONTRADICT" not in labels:
            ids.append(i)
        else:
            banyak_kontra += 1

benchmark_data = train.iloc[ids]

len(corpus), len(abstracts), len(benchmark_data)

(5183, 5183, 370)

In [5]:
# Vector Database (Chroma)
chroma_client = chromadb.PersistentClient()
collection = chroma_client.create_collection(name="scifact_benchmark")

In [12]:
# Upsert preparation
embedder = OpenAIEmbeddings()

metadatas = list()
for i in range(len(corpus)):
    metadatas.append({
        "doc_id" : str(corpus["doc_id"].iloc[i]),
        "title": corpus["title"].iloc[i]
    })

ids = [f"abs{str(i)}" for i in range(len(corpus))]
len(metadatas), metadatas[:2], ids[:2]

(5183,
 [{'doc_id': '4983',
   'title': 'Microstructural development of human newborn cerebral white matter assessed in vivo by diffusion tensor magnetic resonance imaging.'},
  {'doc_id': '5836',
   'title': 'Induction of myelodysplasia by myeloid-derived suppressor cells.'}],
 ['abs0', 'abs1'])

In [13]:
## DANGER
# mahal bos
embeddings = embedder.embed_documents(abstracts)

In [14]:
collection.add(
    embeddings=embeddings,
    documents=abstracts,
    metadatas=metadatas,
    ids=ids
)

In [16]:
docsearch = Chroma(
    client=chroma_client,
    collection_name="scifact_benchmark",
    embedding_function=embedder,
)
print("There are", docsearch._collection.count(), "in the collection")

There are 5183 in the collection


# Retrieval

In [67]:
id = 92
claim = benchmark_data["claim"].iloc[id]
cited_docs = benchmark_data["cited_doc_ids"].iloc[id]
source = corpus[corpus["doc_id"] == cited_docs[0]]

claim, cited_docs

('Energy balance requires hypothalamic glutamate neurotransmission.',
 [45154987, 10534299, 11886686, 25007443, 17150648])

In [68]:
source

,doc_id,title,abstract,structured
4946,45154987,Glutamate mediates the function of melanocorti...,[The melanocortin receptor 4 (MC4R) is a well-...,False


In [79]:
docs_retrieved = docsearch.similarity_search(claim, k=5)

[int(doc.metadata["doc_id"]) for doc in docs_retrieved]

[2617858, 4387784, 11419230, 1744752, 12691537]

# Vanilla Retriever

In [138]:
benchmark_data = benchmark_data.reset_index(drop=True)
benchmark_data = benchmark_data.iloc[:100]
len(benchmark_data)

100

In [139]:
cited_true = list()
cited_pred = list()
for i in tqdm(range(len(benchmark_data))):
    claim = benchmark_data["claim"].iloc[i]
    cited_docs = benchmark_data["cited_doc_ids"].iloc[i]
    cited_true.append(cited_docs)    
    docs_retrieved = docsearch.similarity_search(claim, k=5)
    cited_pred.append([int(doc.metadata["doc_id"]) for doc in docs_retrieved])

100%|██████████| 100/100 [00:35<00:00,  2.85it/s]


In [140]:
def accuracy(cited_true, cited_pred):
    accuracy = 0

    for i in range(len(benchmark_data)):
        accuracy_per_request = 0 

        for j in range(len(cited_true[i])):
            if cited_true[i][j] in cited_pred[i]:
                accuracy_per_request += 1/len(cited_true[i])

        accuracy += accuracy_per_request 

    accuracy = accuracy / len(benchmark_data) * 100

    return accuracy

accuracy(cited_true, cited_pred)

90.10000000000002

# Self-query Retriever

In [122]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

document_content_description = "Collection of scientific papers"
metadata_field_info = [
    AttributeInfo(
        name="doc_id",
        description="Document ID",
        type="string"
    ),
    AttributeInfo(
        name="title",
        description="Title of the scientific paper",
        type="string"
    )
]

retriever = SelfQueryRetriever.from_llm(
    llm,
    docsearch,
    document_content_description,
    metadata_field_info,
    enable_limit=True
)

with get_openai_callback() as cb:
    docs_retrieved = retriever.get_relevant_documents(claim, k=5)
cb

Tokens Used: 1256
	Prompt Tokens: 1214
	Completion Tokens: 42
Successful Requests: 1
Total Cost (USD): $0.02512

# MMR

In [141]:
retriever = docsearch.as_retriever(type="mmr")

cited_true = list()
cited_pred = list()
for i in tqdm(range(len(benchmark_data))):
    claim = benchmark_data["claim"].iloc[i]
    cited_docs = benchmark_data["cited_doc_ids"].iloc[i]
    cited_true.append(cited_docs)    
    docs_retrieved = retriever.get_relevant_documents(claim, k=5)
    cited_pred.append([int(doc.metadata["doc_id"]) for doc in docs_retrieved])

100%|██████████| 100/100 [00:35<00:00,  2.85it/s]


In [142]:
accuracy(cited_true, cited_pred)

84.60000000000002

# Multi-query Retriever

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

llm = OpenAI(model="babbage-002")

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=docsearch.as_retriever(), llm=llm
)

cited_true = list()
cited_pred = list()

with get_openai_callback() as cb:
    for i in range(len(benchmark_data)):
        print(i)
        claim = benchmark_data["claim"].iloc[i]
        cited_docs = benchmark_data["cited_doc_ids"].iloc[i]
        cited_true.append(cited_docs)    
        docs_retrieved = retriever_from_llm.get_relevant_documents(claim, k=5)
        cited_pred.append([int(doc.metadata["doc_id"]) for doc in docs_retrieved])
        print(cb)
        print()

In [152]:
accuracy(cited_true, cited_pred)

79.60000000000002

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

llm = OpenAI(model="davinci-002")

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=docsearch.as_retriever(), llm=llm
)

cited_true = list()
cited_pred = list()

with get_openai_callback() as cb:
    for i in range(len(benchmark_data)):
        print(i)
        claim = benchmark_data["claim"].iloc[i]
        cited_docs = benchmark_data["cited_doc_ids"].iloc[i]
        cited_true.append(cited_docs)    
        docs_retrieved = retriever_from_llm.get_relevant_documents(claim, k=5)
        cited_pred.append([int(doc.metadata["doc_id"]) for doc in docs_retrieved])
        print(cb.total_cost)
        print()

In [155]:
accuracy(cited_true, cited_pred)

78.36666666666669

# Contextual Compression

In [ ]:
# Contextual Compression
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

import warnings
warnings.filterwarnings("ignore")

llm = OpenAI(temperature=0, model="davinci-002")

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=docsearch.as_retriever())

cited_true = list()
cited_pred = list()

with get_openai_callback() as cb:
    for i in range(len(benchmark_data)):
        print(i)
        claim = benchmark_data["claim"].iloc[i]
        cited_docs = benchmark_data["cited_doc_ids"].iloc[i]
        cited_true.append(cited_docs)    
        docs_retrieved = compression_retriever.get_relevant_documents(claim, k=5)
        cited_pred.append([int(doc.metadata["doc_id"]) for doc in docs_retrieved])
        print(cb)
        print()

In [158]:
accuracy(cited_true, cited_pred)

84.60000000000002